In [1]:
import pandas as pd
import numpy as np
import glob, os
import re

# Source

#### basketball-reference.com
https://www.basketball-reference.com/teams/HOU/2017_games.html

- saved regular and playoff seasons from 2008 - 2017 to csv

In [2]:
ls sports_data_raw/NBA/

hou_rockets_plo_08.csv*  hou_rockets_plo_17.csv*  hou_rockets_reg_13.csv*
hou_rockets_plo_09.csv*  hou_rockets_reg_08.csv*  hou_rockets_reg_14.csv*
hou_rockets_plo_13.csv*  hou_rockets_reg_09.csv*  hou_rockets_reg_15.csv*
hou_rockets_plo_14.csv*  hou_rockets_reg_10.csv*  hou_rockets_reg_16.csv*
hou_rockets_plo_15.csv*  hou_rockets_reg_11.csv*  hou_rockets_reg_17.csv*
hou_rockets_plo_16.csv*  hou_rockets_reg_12.csv*


In [3]:
path = 'sports_data_raw/NBA'
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = (pd.read_csv(f) for f in all_files)
df   = pd.concat(df_from_each_file, ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 15 columns):
G             868 non-null int64
Date          868 non-null object
Unnamed: 2    868 non-null object
Unnamed: 3    8 non-null object
Unnamed: 4    868 non-null object
Unnamed: 5    434 non-null object
Opponent      868 non-null object
Unnamed: 7    868 non-null object
Unnamed: 8    55 non-null object
Tm            868 non-null int64
Opp           868 non-null int64
W             868 non-null int64
L             868 non-null int64
Streak        868 non-null object
Notes         1 non-null object
dtypes: int64(5), object(10)
memory usage: 101.8+ KB


In [5]:
df.head()

,G,Date,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes
0,1,"Sat, Apr 19, 2008",8:30p ET,NaN,Box Score,NaN,Utah Jazz,L,NaN,82,93,0,1,L 1,NaN
1,2,"Mon, Apr 21, 2008",8:30p ET,NaN,Box Score,NaN,Utah Jazz,L,NaN,84,90,0,2,L 2,NaN
2,3,"Thu, Apr 24, 2008",8:30p ET,NaN,Box Score,@,Utah Jazz,W,NaN,94,92,1,2,W 1,NaN
3,4,"Sat, Apr 26, 2008",8:30p ET,NaN,Box Score,@,Utah Jazz,L,NaN,82,86,1,3,L 1,NaN
4,5,"Tue, Apr 29, 2008",8:30p ET,NaN,Box Score,NaN,Utah Jazz,W,NaN,95,69,2,3,W 1,NaN


## Create slice with only needed columns

In [6]:
hou_rockets = df[['Date','Unnamed: 2','Unnamed: 5','Opponent','Unnamed: 7','Tm','Opp']]

## Create a column with Houston Rockets

In [7]:
hou_rockets['team'] ='Houston Rockets'

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
hou_rockets = hou_rockets[['Date','Unnamed: 2', 'team','Tm','Unnamed: 5','Opponent','Opp','Unnamed: 7']]

In [9]:
hou_rockets.head()

,Date,Unnamed: 2,team,Tm,Unnamed: 5,Opponent,Opp,Unnamed: 7
0,"Sat, Apr 19, 2008",8:30p ET,Houston Rockets,82,NaN,Utah Jazz,93,L
1,"Mon, Apr 21, 2008",8:30p ET,Houston Rockets,84,NaN,Utah Jazz,90,L
2,"Thu, Apr 24, 2008",8:30p ET,Houston Rockets,94,@,Utah Jazz,92,W
3,"Sat, Apr 26, 2008",8:30p ET,Houston Rockets,82,@,Utah Jazz,86,L
4,"Tue, Apr 29, 2008",8:30p ET,Houston Rockets,95,NaN,Utah Jazz,69,W


## Rename Columns

In [10]:
hou_rockets.rename(columns={
    'Unnamed: 2':'Time',
    'team': 'Team',
    'Tm': 'Team_score',
    'Unnamed: 5': 'Location',
    'Opp': 'Opponent_score',
    'Unnamed: 7': 'Result'
},inplace=True)

In [11]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 19, 2008",8:30p ET,Houston Rockets,82,NaN,Utah Jazz,93,L
1,"Mon, Apr 21, 2008",8:30p ET,Houston Rockets,84,NaN,Utah Jazz,90,L
2,"Thu, Apr 24, 2008",8:30p ET,Houston Rockets,94,@,Utah Jazz,92,W
3,"Sat, Apr 26, 2008",8:30p ET,Houston Rockets,82,@,Utah Jazz,86,L
4,"Tue, Apr 29, 2008",8:30p ET,Houston Rockets,95,NaN,Utah Jazz,69,W


## Inspect Data

In [12]:
hou_rockets.Date.unique()  # need split day

array(['Sat, Apr 19, 2008', 'Mon, Apr 21, 2008', 'Thu, Apr 24, 2008',
       'Sat, Apr 26, 2008', 'Tue, Apr 29, 2008', 'Fri, May 2, 2008',
       'Sat, Apr 18, 2009', 'Tue, Apr 21, 2009', 'Fri, Apr 24, 2009',
       'Sun, Apr 26, 2009', 'Tue, Apr 28, 2009', 'Thu, Apr 30, 2009',
       'Mon, May 4, 2009', 'Wed, May 6, 2009', 'Fri, May 8, 2009',
       'Sun, May 10, 2009', 'Tue, May 12, 2009', 'Thu, May 14, 2009',
       'Sun, May 17, 2009', 'Sun, Apr 21, 2013', 'Wed, Apr 24, 2013',
       'Sat, Apr 27, 2013', 'Mon, Apr 29, 2013', 'Wed, May 1, 2013',
       'Fri, May 3, 2013', 'Sun, Apr 20, 2014', 'Wed, Apr 23, 2014',
       'Fri, Apr 25, 2014', 'Sun, Apr 27, 2014', 'Wed, Apr 30, 2014',
       'Fri, May 2, 2014', 'Sat, Apr 18, 2015', 'Tue, Apr 21, 2015',
       'Fri, Apr 24, 2015', 'Sun, Apr 26, 2015', 'Tue, Apr 28, 2015',
       'Mon, May 4, 2015', 'Wed, May 6, 2015', 'Fri, May 8, 2015',
       'Sun, May 10, 2015', 'Tue, May 12, 2015', 'Thu, May 14, 2015',
       'Sun, May 17, 2015', 'T

In [13]:
hou_rockets.Time.unique()  # need correct time

array(['8:30p ET', '7:30p ET', '7:00p ET', '8:00p ET', '2:30p ET',
       '12:30p ET', '9:30p ET', '10:30p ET', '9:00p ET', '3:30p ET',
       '6:00p ET', '6:30p ET', '1:30p ET', '12:00p ET', '1:00p ET',
       '4:00p ET', '2:00p ET', '3:00p ET', '10:00p ET', '5:30p ET',
       '5:00p ET'], dtype=object)

In [14]:
hou_rockets.Team.unique()  # ok

array(['Houston Rockets'], dtype=object)

In [15]:
hou_rockets.Team_score.unique()  # ok

array([ 82,  84,  94,  95,  91, 108, 103,  86,  89,  77,  92, 100,  98,
        99,  78,  70, 102, 101, 105, 107, 120, 121, 118, 111, 130, 109,
       115, 124, 119, 113, 106,  80, 128,  90,  97,  81, 126,  96, 125,
        75, 104,  85,  88,  83, 116,  93, 112, 110, 117,  74,  69,  79,
       131, 114,  87,  73,  66, 127, 129, 123, 140, 122, 136, 134, 145,
       132, 137, 133, 142, 139, 135])

In [16]:
hou_rockets.Location.unique()  # 

array([nan, '@'], dtype=object)

In [17]:
hou_rockets.Opponent.unique()  # ok

array(['Utah Jazz', 'Portland Trail Blazers', 'Los Angeles Lakers',
       'Oklahoma City Thunder', 'Dallas Mavericks',
       'Los Angeles Clippers', 'Golden State Warriors',
       'San Antonio Spurs', 'Milwaukee Bucks', 'Charlotte Bobcats',
       'Memphis Grizzlies', 'Phoenix Suns', 'Miami Heat',
       'Denver Nuggets', 'Sacramento Kings', 'New Jersey Nets',
       'Toronto Raptors', 'Philadelphia 76ers', 'Detroit Pistons',
       'Orlando Magic', 'Chicago Bulls', 'Boston Celtics',
       'New York Knicks', 'Washington Wizards', 'Minnesota Timberwolves',
       'New Orleans Hornets', 'Seattle SuperSonics', 'Indiana Pacers',
       'Cleveland Cavaliers', 'Atlanta Hawks', 'Brooklyn Nets',
       'New Orleans Pelicans', 'Charlotte Hornets'], dtype=object)

In [18]:
hou_rockets.Opponent.value_counts(dropna=False)  # ok

Portland Trail Blazers    47
San Antonio Spurs         46
Golden State Warriors     46
Dallas Mavericks          44
Oklahoma City Thunder     44
Los Angeles Clippers      42
Los Angeles Lakers        42
Utah Jazz                 41
Memphis Grizzlies         40
Denver Nuggets            37
Phoenix Suns              36
Sacramento Kings          36
Minnesota Timberwolves    36
New Orleans Hornets       23
Washington Wizards        20
Toronto Raptors           20
New York Knicks           19
Milwaukee Bucks           19
Chicago Bulls             19
Atlanta Hawks             19
Orlando Magic             19
Detroit Pistons           19
Cleveland Cavaliers       19
Philadelphia 76ers        19
Indiana Pacers            19
Boston Celtics            19
Miami Heat                19
New Orleans Pelicans      16
Charlotte Bobcats         14
Brooklyn Nets             10
New Jersey Nets            9
Charlotte Hornets          6
Seattle SuperSonics        4
Name: Opponent, dtype: int64

In [19]:
hou_rockets.Opponent_score.unique()  # ok

array([ 93,  90,  92,  86,  69, 113,  81, 107,  83,  88,  76, 111, 108,
        87, 118,  80,  89, 120, 105, 104, 103, 100, 122, 112, 116, 123,
        98,  99, 128, 121,  94, 117, 109, 124, 110, 115,  96, 114,  95,
        82,  71,  77,  97,  79,  91,  84,  85,  73,  75, 106,  66, 102,
       101,  78,  74, 130, 125, 133, 119, 132,  70, 134, 137,  65, 131,
       126, 140, 127])

In [20]:
hou_rockets.Result.unique()  # ok

array(['L', 'W'], dtype=object)

## Location Column

- change @ to away, nan to home

In [21]:
hou_rockets.Location.value_counts(dropna=False)

@      434
NaN    434
Name: Location, dtype: int64

In [22]:
hou_rockets['Location'] = np.where(hou_rockets['Location'] =='@', 'away', 'home')

In [23]:
hou_rockets.Location.value_counts(dropna=False)

away    434
home    434
Name: Location, dtype: int64

In [24]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 19, 2008",8:30p ET,Houston Rockets,82,home,Utah Jazz,93,L
1,"Mon, Apr 21, 2008",8:30p ET,Houston Rockets,84,home,Utah Jazz,90,L
2,"Thu, Apr 24, 2008",8:30p ET,Houston Rockets,94,away,Utah Jazz,92,W
3,"Sat, Apr 26, 2008",8:30p ET,Houston Rockets,82,away,Utah Jazz,86,L
4,"Tue, Apr 29, 2008",8:30p ET,Houston Rockets,95,home,Utah Jazz,69,W


## Date column

- split Date:
    - Day : Sun
    - Date : Month, day, Year

In [25]:
# hou_rockets['Day'] = hou_rockets['Date'].str.split(',').apply(lambda x: x[0])
# hou_rockets['Date'] = hou_rockets['Date'].str.split(',').apply(lambda x: x[1:])

In [26]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,"Sat, Apr 19, 2008",8:30p ET,Houston Rockets,82,home,Utah Jazz,93,L
1,"Mon, Apr 21, 2008",8:30p ET,Houston Rockets,84,home,Utah Jazz,90,L
2,"Thu, Apr 24, 2008",8:30p ET,Houston Rockets,94,away,Utah Jazz,92,W
3,"Sat, Apr 26, 2008",8:30p ET,Houston Rockets,82,away,Utah Jazz,86,L
4,"Tue, Apr 29, 2008",8:30p ET,Houston Rockets,95,home,Utah Jazz,69,W


In [27]:
#day = hou_rockets['Date'].str.split(',').apply(lambda x: x[0])

In [28]:
hou_rockets['Date'] = pd.to_datetime(hou_rockets['Date'])

In [29]:
hou_rockets.head()

,Date,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
0,2008-04-19,8:30p ET,Houston Rockets,82,home,Utah Jazz,93,L
1,2008-04-21,8:30p ET,Houston Rockets,84,home,Utah Jazz,90,L
2,2008-04-24,8:30p ET,Houston Rockets,94,away,Utah Jazz,92,W
3,2008-04-26,8:30p ET,Houston Rockets,82,away,Utah Jazz,86,L
4,2008-04-29,8:30p ET,Houston Rockets,95,home,Utah Jazz,69,W


In [30]:
hou_rockets = hou_rockets.set_index('Date').sort_index(ascending=True)

In [31]:
hou_rockets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 868 entries, 2007-10-30 to 2017-05-11
Data columns (total 7 columns):
Time              868 non-null object
Team              868 non-null object
Team_score        868 non-null int64
Location          868 non-null object
Opponent          868 non-null object
Opponent_score    868 non-null int64
Result            868 non-null object
dtypes: int64(2), object(5)
memory usage: 54.2+ KB


In [32]:
hou_rockets.head()

,Time,Team,Team_score,Location,Opponent,Opponent_score,Result
Date,,,,,,,
2007-10-30,7:30p ET,Houston Rockets,95,away,Los Angeles Lakers,93,W
2007-11-01,7:00p ET,Houston Rockets,106,away,Utah Jazz,95,W
2007-11-03,7:30p ET,Houston Rockets,89,home,Portland Trail Blazers,80,W
2007-11-05,7:30p ET,Houston Rockets,98,away,Dallas Mavericks,107,L
2007-11-06,7:30p ET,Houston Rockets,89,home,San Antonio Spurs,81,W


## Save clean data to  to csv

In [33]:
hou_rockets.to_csv('sports_data_clean/hou_rockets_08_17_clean.csv')

In [34]:
ls sports_data_clean/

astros_08_17_clean.csv*  dynamo_08_17_clean.csv*  hou_rockets_08_17_clean.csv*


In [35]:
## Finish